In [1]:
import sys
sys.path.insert(1, '../')
import torch
from helpers.data import get_dataloaders
from helpers.train import TrainingManager
from helpers.loss_accuracy import accuracy
import models.resnet
import torch
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from tqdm.auto import tqdm
import numpy as np
import h5py

In [2]:
device = torch.device('cuda')
no_iterations = 3200
batch_size = 1000
file_name = '/media/data1/idokessler/cifar10/pretrained_resnet110_wo_last_layer_outputs.hdf5'

In [3]:
model = models.resnet.resnet110()
model.load_state_dict(torch.load('../pretrained_models/resnet110-1d1ed7c2_new.th'))

<All keys matched successfully>

In [4]:
new_model = torch.nn.Sequential(*[model.pre_layer, model.layer1, model.layer2, *model.layer3[:17]]).cuda()

In [5]:
trainloader, testloader = get_dataloaders('cifar10', batch_size)

Files already downloaded and verified


In [6]:
def cycle(iterable):
    while True:
        for i in iterable:
            yield i
def iterate_for_x(iterable, no):
    it = iter(cycle(iterable))
    for i in range(no):
        yield next(it)

array([1764001,  837219, 2312052,  945388, 1175600,  758575, 2889801,
        959632, 1851849, 3091072, 2445675, 1937069, 1975764, 3100104,
       1066804,  730726, 1887027, 1171742, 1769763, 1302360, 1501676,
        645417,   85244,  439225,  586766,  958142, 1373435, 1176196,
         16806, 1585536, 2148580, 1370186, 2428559,   58314, 1466988,
       1763684, 1184695, 2149846,   78040, 3122406,  293437, 3025746,
       1501347, 1379029, 2550899,  964272,  824223, 1945370, 2782095,
       2660194, 2113554, 1093418, 2129746, 2759904, 1072870,  826150,
        121548, 2306344, 1700552, 1614474, 1535503,  730401, 2930459,
       2967105, 2525108,  761620, 3111513, 2881886, 2793105, 1483210,
       1934355, 1750418,   72840, 2399094,  324522, 3146845, 2064944,
       3015435,  329642, 1144050, 2073195,  230099,  192574,  257687,
       2398238, 1298279, 1994921, 2940332, 1109502,  892189, 1238937,
       2111568, 2900050, 1805388, 2222980, 1453229, 1254889, 2586020,
       2957348, 1308

In [33]:
for i in tqdm(range(10000)):
    x = f['test_x'][np.unique(np.random.randint(0,10000,(100)))]

KeyboardInterrupt: 

In [15]:
all(f['test_y'][:] < 10)

True

In [8]:
f.create_dataset('train_x', (3200000 ,64, 8, 8), np.float32)
f.create_dataset('train_y', (3200000,), np.int)
f.create_dataset('test_x', (10000, 64, 8, 8), np.float32)
f.create_dataset('test_y', (10000,), np.int)

<HDF5 dataset "test_y": shape (10000,), type "<i8">

In [9]:
new_model.eval()
with torch.no_grad():
    for i,(x,y) in enumerate(tqdm(testloader)):
        f['test_x'][i*batch_size:(i+1)*batch_size] = new_model(x.cuda()).cpu()
        f['test_y'][i*batch_size:(i+1)*batch_size] = y
    for i, (x, y) in enumerate(tqdm(iterate_for_x(trainloader, no_iterations), total=no_iterations)):
        f['train_x'][i*batch_size:(i+1)*batch_size] = new_model(x.cuda()).cpu()
        f['train_y'][i*batch_size:(i+1)*batch_size] = y

In [16]:
f.close()